# Main Code Space

# 1. Data Loading

In [ ]:
from functions_new import *
import matplotlib.pyplot as plt
import numpy as np
from sklearn.metrics import classification_report, confusion_matrix
import seaborn as sns
import tensorflow as tf
import sys
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout, Conv1D, BatchNormalization, ReLU, GlobalAveragePooling1D, MaxPooling1D 
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import layers, models

# WindowSize            : 10(0.8565), 20(0.9240), 30(0.9488), 40(0.9584), 50(0.9636)
# Epoch(window=20)      : 20(0.9448), 30(0.9564)

# {Mode, windowSize, Stride, Epoch} 
# LSTM : {3, 20, 1, 20} -> 0.9573, {4, 20, 1, 20} -> 0.9606, {4, 10, 1, 20} -> 0.9500, {4, 30, 1, 20} -> 0.9685, {3, 30, 10, 20} -> 0.9303 (Not Bad!!, 학습빠름)
# LSTM + CNN : {3, 50, 5, 20} -> 0.9550(결과 Good), {3, 50, 1, 20} -> 0.9721, {3, 30, 1, 20} -> 0.9606, {3, 20, 1, 20} -> 별로임, {3, 40, 1, 20} -> 0.9668, {3, 50, 1, 20} -> 0.9723(결과 별로임)


# MODE = 0            # ALL_LABEL_ALL_SENSOR
# MODE = 1            # ALL_LABEL_LOWER_SENSOR
# MODE = 2            # ALL_LABEL_UPPER_SENSOR
MODE = 3            # Transient_LABEL_ALL_SENSOR
# MODE = 4            # Transient_LABEL_ALL_SENSOR_NO_STOPWALKING
# MODE = 5            # Transient_LABEL_LOWER_SENSOR
# MODE = 6            # Transient_LABEL_UPPER_SENSOR
# MODE = 7            # Add "Start Walking" 

# Model = 0           # LSTM  (0.9333)
# Model = 1           # CNN (0.9120)
# Model = 2           # TCN (0.9133)
# Model = 3           # LSTM + CNN (0.9593)
Model = 4           # CNN + LSTM 
# Model = 5           # Transformer


folder_paths = ['../Data/data_250612', '../Data/data_250616', '../Data/data_250618']
all_trials = []

for folder_path in folder_paths:
    trials = Load_files(folder_path, pattern='cropped_*.h5')
    trials = [trial for trial in trials if 'test' not in trial['file']]     # Temporary remove Test set
    all_trials.extend(trials)

## Labeling ##
for trial in all_trials:
    trial['label'] = Generate_labels(trial)
    Convert_labels_to_int(trial, phase_to_int)

## EMG filtering ##
all_trials = process_all_emg(all_trials, lpf_fc=3, norm_method='max')

trial_num = len(all_trials)
print(f"Total trials: {trial_num}")